In [1]:
from collections import defaultdict
import json
import ee
import pandas as pd
import geopandas as gpd
import geemap

In [2]:
ee.Authenticate()

Enter verification code: 4/1Adeu5BWy2EPddJV4MEh03FJ-6ltLQUE4pitJ70A7iV9hNsots7iRiQCKSFA

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [3]:
urbext_2020 = ee.FeatureCollection('projects/wri-datalab/SCL-Cities/urbanextents__bycountry_2020')
urbext_data_2020 = geemap.ee_to_pandas(urbext_2020)

In [4]:
worldpop_2020 = ee.ImageCollection("WorldPop/GP/100m/pop").filter(ee.Filter.equals('year', 2020))

In [5]:
popcodes = worldpop_2020.aggregate_array('country').getInfo()

In [6]:
scale = 100
def doOneYear(ufc, upd, pop_ic, year):
    urbarea_res = defaultdict(list)
    pop_res = defaultdict(list)
    print(year)
    for i in range(len(upd)):
        ua = upd.iloc[i]
        ua_f = ufc.filter(ee.Filter.eq('city_ids', str(ua['city_ids'])))
        geom = ua_f.geometry()
        if ua['country'] in popcodes:
            local_area = ee.Image.pixelArea().reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).get('area').getInfo()
            localpop_img = pop_ic.filter(ee.Filter.equals('country', ua['country'])).select('population').first()
            localpop = localpop_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).get('population').getInfo()
            urbarea_res[ua['country']].append(local_area)
            pop_res[ua['country']].append(localpop)
        print(i, end=' ')
    return urbarea_res, pop_res

urbaeares_2020, popres_2020 = doOneYear(urbext_2020, urbext_data_2020, worldpop_2020, 2020)

out_dicts = []
country_list = list(urbaeares_2020.keys())
country_list.sort()
for country in country_list:
    out_dicts.append(
        {
            'country': country,
            'urban_area_2020': sum(urbaeares_2020[country]),
            'urban_population_2020': sum(popres_2020[country])
        }
    )
res_pd = pd.DataFrame(out_dicts)
res_pd.to_csv('CTY-1_popdensity.csv')

2020
0 1 2 {'area': 51093686.05910313}


NameError: name 'cow' is not defined